In [1]:
import os

import GPRutils
import DESutils
import vK2KGPR

import numpy as np

import warnings
warnings.filterwarnings("ignore")

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


Compare how well the model does with the fitCorr params vs with the final parameters from the GP optimization.

In [2]:
def compareParamsBeforeAfter(directory, expNum, useRMS, curl, file):
    ext = ""
    if useRMS:
        ext += "R"
    if curl:
        ext += "C"
    dataC = GPRutils.loadFITS(os.path.join(directory, f"{expNum}", f"GPR.{expNum}.?{ext}.fits"))
    
    dC1 = GPRutils.dataContainer()
    dC1.load(expNum, useRMS=useRMS)
    GP1 = vK2KGPR.vonKarman2KernelGPR(dC1, curl=curl, printing=True)
    GP1.fit(dataC.fitCorrParams)
    dC1.postFitCorr_sigmaClip(GP1)
    dC1.JackKnife(GP1)

    dC2 = GPRutils.dataContainer()
    dC2.load(expNum, useRMS=useRMS)
    GP2 = vK2KGPR.vonKarman2KernelGPR(dC2, curl=curl, printing=True)
    GP2.fit(dataC.params)
    dC2.postFitCorr_sigmaClip(GP2)
    dC2.JackKnife(GP2)

    line = "----------fitCorr Params----------"
    xi0, xif = dC1.JackKnifeXi()
    xi0, Xerr, Yerr = xi0
    line += f"\nxi0: {xi0:.3f} ± {np.sqrt(Xerr**2 + Yerr**2):.3f} mas^2"
    xif, Xerr, Yerr = xif
    line += f"\nxif: {xif:.3f} ± {np.sqrt(Xerr**2 + Yerr**2):.3f} mas^2"
    line += f"\nReduction: {xi0/xif:.3f}\n"
    with open(file, mode="a+") as f:
        f.write(line)
    GPRutils.printParams(dataC.fitCorrParams, header=True, file=file)
    GPRutils.printParams(dataC.fitCorrParams, file=file)
    
    line = "----------Optimize Params----------"
    xi0, xif = dC2.JackKnifeXi()
    xi0, Xerr, Yerr = xi0
    line += f"\nxi0: {xi0:.3f} ± {np.sqrt(Xerr**2 + Yerr**2):.3f} mas^2"
    xif, Xerr, Yerr = xif
    line += f"\nxif: {xif:.3f} ± {np.sqrt(Xerr**2 + Yerr**2):.3f} mas^2"
    line += f"\nReduction: {xi0/xif:.3f}\n"
    with open(file, mode="a+") as f:
        f.write(line)
    GPRutils.printParams(dataC.params, header=True, file=file)
    GPRutils.printParams(dataC.params, file=file)
    with open(file, mode="a+") as f:
        f.write("\n\n\n")
    
exps = DESutils.findExpNums()

In [ ]:
directory = "/home/fortino/test_N"
for exp in exps[:5]:
    try:
        compareParamsBeforeAfter(directory, exp, False, False, os.path.join(directory, "Ncompare.out"))
    except np.linalg.LinAlgError:
         continue

Exposure: 348819
Band: z
Number of objects: 116592
INFO: Query finished. [astroquery.utils.tap.core]
There were 10207 matches within 0.5 arcsec.
Exposure: 348819
Band: z
Number of objects: 116592
INFO: Query finished. [astroquery.utils.tap.core]
There were 10207 matches within 0.5 arcsec.
K Variance     Outer Scale    Diameter       Wind X         Wind Y         
109.1565559    1.5349095      -0.0061141     0.0890528      -0.0355237     
K Variance     Outer Scale    Diameter       Wind X         Wind Y         
124.0552915    2.0130231      -0.0090629     -0.0091152     -0.0138322     
Exposure: 355303
Band: z
Number of objects: 107591
INFO: Query finished. [astroquery.utils.tap.core]
There were 11829 matches within 0.5 arcsec.
Exposure: 355303
Band: z
Number of objects: 107591
INFO: Query finished. [astroquery.utils.tap.core]
There were 11829 matches within 0.5 arcsec.
K Variance     Outer Scale    Diameter       Wind X         Wind Y         
118.5280309    1.5684838      0.0192712 

In [ ]:
directory = "/home/fortino/test_R"
for exp in exps[:5]:
    try:
        compareParamsBeforeAfter(directory, exp, True, False, os.path.join(directory, "Rcompare.out"))
    except np.linalg.LinAlgError:
         continue

In [ ]:
directory = "/home/fortino/test_R"
for exp in exps[:5]:
    try:
        compareParamsBeforeAfter(directory, exp, False, False, os.path.join(directory, "Rcompare.out"))
    except np.linalg.LinAlgError:
         continue

In [ ]:
directory = "/home/fortino/test_C"
for exp in exps[:5]:
    try:
        compareParamsBeforeAfter(directory, exp, False, True, os.path.join(directory, "Ccompare.out"))
    except np.linalg.LinAlgError:
         continue

In [ ]:
directory = "/home/fortino/test_RC"
for exp in exps[:5]:
    try:
        compareParamsBeforeAfter(directory, exp, True, True, os.path.join(directory, "RCcompare.out"))
    except np.linalg.LinAlgError:
         continue